In [31]:
import numpy as np
from ase.io import read
from calorine.calculators import CPUNEP
from calorine.tools import get_force_constants, relax_structure
from matplotlib import pyplot as plt
from matplotlib import gridspec
from pandas import DataFrame
import os
import warnings
warnings.filterwarnings("ignore", message="logm result may be inaccurate*")

In [32]:
plt.rcParams.update({
    'font.size': 18,  
    'axes.labelsize': 18,  
    'xtick.labelsize': 16,  
    'ytick.labelsize': 16,  
    'legend.fontsize': 14  
})

In [33]:
# Ensure output directory exists
os.makedirs('DOS', exist_ok=True)

In [34]:
def interpolate_k_path(kpoints, points_per_segment):
    """Interpolate k-point path between high-symmetry points."""
    k_path = []
    for i in range(len(kpoints) - 1):
        start = np.array(kpoints[i])
        end = np.array(kpoints[i + 1])
        # Linear interpolation between k-points
        k_path.extend((1 - t) * start + t * end for t in np.linspace(0, 1, points_per_segment, endpoint=False))
    k_path.append(kpoints[-1])
    return np.array(k_path)

def prepare_custom_path(kpoints, num_points):
    """Prepare custom k-point path and its linear coordinates for plotting."""
    k_path = interpolate_k_path(kpoints, num_points)
    linearcoord = [0]
    for i in range(1, len(k_path)):
        delta_k = np.linalg.norm(k_path[i] - k_path[i - 1])
        linearcoord.append(linearcoord[-1] + delta_k)

    custom_labels = ['Γ', 'X']
    return {
        'explicit_kpoints_rel': k_path,
        'explicit_kpoints_labels': custom_labels,
        'explicit_kpoints_linearcoord': linearcoord
    }

In [35]:
# Define custom high-symmetry path (Γ → X)
custom_kpoints = [
    [0, 0, 0],     # Γ point
    [0.0, 0, 0.5], # X point (along [001])
]
num_points_per_segment = 50
custom_path = prepare_custom_path(custom_kpoints, num_points_per_segment)

In [36]:
#structure_file = "Structure/Si_wire_1.09nm.xyz"
#structure_file = "Structure/Si_wire_1.63nm.xyz"
#structure_file = "Structure/Si_wire_2.17nm.xyz"
structure_file = "Structure/Si_wire_3.26nm.xyz"
#structure_file = "Structure/Si_wire_5.43nm.xyz"
#structure_file = "Structure/Si_wire_10.86nm.xyz"
structure = read(structure_file)

In [37]:
# Attach NEP calculator
calculator = CPUNEP('NEP/Si_2022_NEP3_3body.txt')
structure.calc = calculator

In [38]:
# Relax the atomic structure to minimize forces
relax_structure(structure, fmax=0.0001)

# Calculate second-order force constants (supercell = 1x1x4)
phonon = get_force_constants(structure, calculator, [1, 1, 4])

In [39]:
# Calculate phonon DOS using mesh sampling
phonon.run_mesh([1, 1, 600])
phonon.run_total_dos(freq_min=0, freq_max=15, freq_pitch=0.01)
dos = phonon.get_total_dos_dict()

# Normalize the DOS
N_atoms = 8 / (5.431e-10)**3  # atoms/m³
total_modes = 3 * N_atoms     # 3N phonon modes

frequencies = dos['frequency_points'] * 2 * np.pi
raw_dos = dos['total_dos']

# Calculate area under raw DOS curve (rad/s unit)
area = np.trapezoid(raw_dos, frequencies) * 1e12

# Normalize DOS so that integral = 3N
normalized_dos = raw_dos * (total_modes / area)

# Save normalized DOS to file
dos_data = np.column_stack((frequencies, normalized_dos))
dos_filename = f"DOS/DOS_{os.path.basename(structure_file).replace('.xyz', '')}.txt"
np.savetxt(dos_filename, dos_data, header='Frequency(rad/ps) Normalized_DOS', fmt='%.6f')
print(f"Saved normalized DOS to {dos_filename}")

Saved normalized DOS to DOS/DOS_Si_wire_3.26nm.txt


In [40]:
# Compute and plot band structure only for a specific file

if structure_file == "Structure/Si_wire_1.09nm.xyz":
    # Run phonon band structure calculation
    phonon.run_band_structure([custom_path['explicit_kpoints_rel']])
    band = phonon.get_band_structure_dict()

    # Convert frequency to rad/ps and format into DataFrame
    df = DataFrame(band['frequencies'][0]) * 2 * np.pi
    df.index = custom_path['explicit_kpoints_linearcoord']

    # Setup figure layout: dispersion + DOS side by side
    fig = plt.figure(figsize=(6, 4))
    spec = gridspec.GridSpec(nrows=1, ncols=2, width_ratios=[4, 1], wspace=0.05)

    # === Plot phonon dispersion curve ===
    ax_disp = fig.add_subplot(spec[0])
    for col in df.columns:
        ax_disp.plot(df.index, df[col], color='darkorange')

    ax_disp.set_xlim(df.index.min(), df.index.max())
    ax_disp.set_ylabel('Frequency (rad/ps)')

    # Mark high-symmetry points on x-axis
    xticks_positions = [custom_path['explicit_kpoints_linearcoord'][i] for i in [0, 50]]
    xticks_labels = ['Γ', 'X']
    ax_disp.set_xticks(xticks_positions)
    ax_disp.set_xticklabels(xticks_labels)

    for x in xticks_positions:
        ax_disp.axvline(x=x, color='gray', linestyle='--', linewidth=1)

    # === Plot DOS on the right ===
    ax_dos = fig.add_subplot(spec[1], sharey=ax_disp)
    ax_dos.plot(normalized_dos / 1e16, frequencies, color='darkorange')  # Scaled for visual clarity
    ax_dos.set_xlabel('DOS')
    ax_dos.tick_params(labelleft=False)  # Hide y-axis ticks on DOS panel
    ax_dos.set_xlim(-0.05, 1.05)

    # Save and display the figure
    output_filename = f"Fig/Disp/Disp_DOS_{os.path.basename(structure_file).replace('.xyz', '')}.png"
    plt.savefig(output_filename, dpi=300, bbox_inches='tight')
    plt.show()